In [20]:
import math
import os
from collections import defaultdict
import pandas as pd
import time

# SCHEMACONATCTS

In [21]:
class File:
    
    "Read every lines from pdb format file"
    
    def __init__(self,fileName):
        self.fileName=fileName


    
    def read(self):
        f=open(self.fileName,"r")
        lines=f.readlines()
        atom_lines=[]
        for line in lines:
            if line[0:4]=="ATOM":
                atom_lines.append(line)
        return atom_lines

In [22]:
class Atom:

    "Reads a PDB file ATOM line and extracts fields \
        according to the pdb format"

    def __init__(self, line):
        self.line = line
        self.recordName = self.getRecordName()
        self.atomNum = self.getAtomNum()
        self.atomName = self.getAtomName()
        self.altLoc = self.getaltLoc()
        self.resName = self.getResName()
        self.chainID = self.getChainId()
        self.resNum = self.getResNum()
        self.icode = self.getiCode()
        self.X = self.getCoordX()
        self.Y = self.getCoordY()
        self.Z = self.getCoordZ()
        self.occupancy = self.getOccupancy()
        self.tempFactor = self.getTempFactor()
        self.element = self.getElementSymbol()
        self.charge = self.getCharge()

    def getRecordName(self):
        return self.line[0:6].strip()

    def getAtomNum(self):
        return int(self.line[6:11].strip())

    def getAtomName(self):
        return self.line[12:16].strip()

    def getaltLoc(self):
        "Alternate location indicator."
        return self.line[16]

    def getResName(self):
        return self.line[17:20].strip()

    def getChainId(self):
        return self.line[21].strip()

    def getResNum(self):
        return self.line[22:26].strip()

    def getiCode(self):
        "Code for insertion of residues."
        return self.line[26]

    def getCoordX(self):
        return float(self.line[30:38].strip())

    def getCoordY(self):
        return float(self.line[38:46].strip())

    def getCoordZ(self):
        return float(self.line[46:54].strip())

    def getOccupancy(self):
        return float(self.line[54:60].strip())

    def getTempFactor(self):
        return float(self.line[60:66].strip())

    def getElementSymbol(self):
        return self.line[76:78].strip()

    def getCharge(self):
        return self.line[78:80].strip()

    def getDistance(self, atom):
        return math.sqrt((self.X-atom.X)**2+(self.Y-atom.Y)**2+(self.Z-atom.Z)**2)


In [23]:
class Residues:

    def __init__(self,lines):
        Res = defaultdict(list)
        self.res=Res
        self.lines=lines
        if lines:
            i=0
            for j in range(i,len(lines)):
                atom1=Atom(lines[i])
                self.resName=atom1.resName
                self.resNum=atom1.resNum
                self.chainID=atom1.chainID
                atom=Atom(lines[j])
                if atom.resNum==self.resNum:
                    self.res[int(self.resNum)].append(atom)
                else:
                    i=j
        else:
            atom=None
            self.resName='-'
            self.resNum=-1
            self.chainID=None
            self.atoms=[]
    
    def sequence(self,chainId=["A",""]):
        
        '''
        ouput the amino acid sequence
        '''

        trans = {
        'ALA':'A','CYS':'C','ASP':'D','GLU':'E',
        'PHE':'F','GLY':'G','HIS':'H','LYS':'K',
        'ILE':'I','LEU':'L','MET':'M','ASN':'N',
        'PRO':'P','GLN':'Q','ARG':'R','SER':'S',
        'THR':'T','VAL':'V','TYR':'Y','TRP':'W',"-":"-",
        }
        return ''.join(trans[Atom(self.lines[i]).resName] for i in range(len(self.lines)) if (Atom(self.lines[i-1]).resNum!=Atom(self.lines[i]).resNum) and Atom(self.lines[i]).chainID in chainId)

        
    def getDistance(self,min_dist=4.5):
        '''1) Calculate the minimum distance between two atoms from different residues (without H and O) \n \
           2) Judge the contact condition between residues according to the min_dist and 1) calculated distances  \n \
           3) return a Dataframe
        '''
        temp={}
        dist_all=pd.DataFrame()
        atom_to_exclude=['H',"O"]
        n=0
        for i in range(int(Atom(self.lines[0]).resNum),int(Atom(self.lines[-1]).resNum)):
            for j in range(i+1,int(Atom(self.lines[-1]).resNum)+1):
                min_distance=1e6
                for atom1 in self.res[i]:
                    for atom2 in self.res[j]:
                        if atom1.atomName in atom_to_exclude or atom2.atomName in atom_to_exclude:
                            continue
                        dist=atom1.getDistance(atom2)
                        if dist<min_distance:
                            min_distance=dist
                            name=str(atom1.resNum)+"-"+str(atom2.resNum)
                            i_reindex=i-int(Atom(self.lines[0]).resNum)+1
                            j_reindex=j-int(Atom(self.lines[0]).resNum)+1
                            temp[n]=[i,j,i_reindex,j_reindex,name,dist]
                n=n+1
        dist_all=pd.DataFrame(temp).T
        dist_all.columns=["i","j","i_reindex","j_reindex","pairs","Distance"]
        dist_all.insert(6,"isContact","False")
        for i in range(len(dist_all.index)):
            if dist_all.iloc[i].values[5]<min_dist:
                dist_all.iloc[[i],[6]]="True"
        return dist_all

        

In [24]:
dist=Residues(File("1JPZ.pdb").read()).getDistance()
dist_true=dist[dist['isContact']=='True'].sort_values(by=['isContact','i']).reset_index(drop=True)
dist

,i,j,i_reindex,j_reindex,pairs,Distance,isContact
0,1,2,1,2,1-2,2.45121,True
1,1,3,1,3,1-3,4.79223,False
2,1,4,1,4,1-4,8.32006,False
3,1,5,1,5,1-5,10.8584,False
4,1,6,1,6,1-6,12.4237,False
...,...,...,...,...,...,...,...
104648,455,457,455,457,455-457,5.33683,False
104649,455,458,455,458,455-458,7.41022,False
104650,456,457,456,457,456-457,2.40999,True
104651,456,458,456,458,456-458,5.14067,False


# SCHEMADisruption

In [25]:
dist.insert(7,"C_kl",0)
dist

,i,j,i_reindex,j_reindex,pairs,Distance,isContact,C_kl
0,1,2,1,2,1-2,2.45121,True,0
1,1,3,1,3,1-3,4.79223,False,0
2,1,4,1,4,1-4,8.32006,False,0
3,1,5,1,5,1-5,10.8584,False,0
4,1,6,1,6,1-6,12.4237,False,0
...,...,...,...,...,...,...,...,...
104648,455,457,455,457,455-457,5.33683,False,0
104649,455,458,455,458,455-458,7.41022,False,0
104650,456,457,456,457,456-457,2.40999,True,0
104651,456,458,456,458,456-458,5.14067,False,0


In [26]:
def get_disruption(dist,w=14): 
    start_time=time.time()
    disruption={}
    x=range(dist["i_reindex"].iloc[0],dist["j_reindex"].iloc[-1])
    for i in x:
        disruption[i]=0
        for j in range(i-w+1,i+1):
            for k in range(j,j+w-1):
                for l in range(k+1,j+w):
                    try:
                        if k > 0 and l > 0:
                            if dist.loc[(dist['i_reindex']==k)&(dist['j_reindex']==l),"isContact"].values[0]=="True":
                                dist.loc[(dist['i_reindex']==k)&(dist['j_reindex']==l),"C_kl"]=1
                            else:
                                dist.loc[(dist['i_reindex']==k)&(dist['j_reindex']==l),"C_kl"]=0
                            disruption[i]=disruption[i]+ dist.loc[(dist['i_reindex']==k)&(dist['j_reindex']==l),"C_kl"].values[0]
                    except IndexError:
                        pass
        disruption[i]=(1/math.sqrt(w))*disruption[i]
                    
        num=i*50//len(x)
        process="\r[w=%s%4s%%]:|%-50s|\n" % (w,2*num,'#'*num)
        print(process,end='',flush=True)
        
    end_time=time.time()
    S_profile=pd.DataFrame(data=disruption,index=["S"]).T
    S_profile.to_csv("S_profile(W={}).csv".format(w))
    fig=S_profile.plot(title="S_profile(W={}) t={:.3f}min".format(w,(end_time-start_time)/60),figsize=(20,10))
    fig.figure.savefig("S_profile(W={}).jpg".format(w))
    
    

In [28]:
get_disruption(dist,w=5)


[w=5   0%]:|                                                  |
[w=5   0%]:|                                                  |
[w=5   0%]:|                                                  |
[w=5   0%]:|                                                  |
[w=5   0%]:|                                                  |
[w=5   0%]:|                                                  |
[w=5   0%]:|                                                  |
[w=5   0%]:|                                                  |
[w=5   0%]:|                                                  |
[w=5   2%]:|#                                                 |
[w=5   2%]:|#                                                 |
[w=5   2%]:|#                                                 |
[w=5   2%]:|#                                                 |
[w=5   2%]:|#                                                 |
[w=5   2%]:|#                                                 |
[w=5   2%]:|#                           

KeyboardInterrupt: 

# Example

In [8]:
m=Residues(File("1JPZ.pdb").read())

In [12]:
m.sequence(chainId=['A',"B"])

'TIKEMPQPKTFGELKNLPLLNTDKPVQALMKIADELGEIFKFEAPGRVTRYLSSQRLIKEACDESRFDKNLSQALKFVRDFAGDGLFTSWTHEKNWKKAHNILLPSFSQQAMKGYHAMMVDIAVQLVQKWERLNADEHIEVPEDMTRLTLDTIGLCGFNYRFNSFYRDQPHPFITSMVRALDEAMNKLQRANPDDPAYDENKRQFQEDIKVMNDLVDKIIADRKASQSDDLLTHMLNGKDPETGEPLDDENIRYQIITFLIAGHETTSGLLSFALYFLVKNPHVLQKAAEEAARVLVDPVPSYKQVKQLKYVGMVLNEALRLWPTAPAFSLYAKEDTVLGGEYPLEKGDELMVLIPQLHRDKTIWGDDVEEFRPERFENPSAIPQHAFKPFGNGQRACIGQQFALHEATLVLGMMLKHFDFEDHTNYELDIKETLTLKPEGFVVKAKSKKIPLGGITIKEMPQPKTFGELKNLPLLNTDKPVQALMKIADELGEIFKFEAPGRVTRYLSSQRLIKEACDESRFDKNLSQALKFVRDFAGDGLFTSWTHEKNWKKAHNILLPSFSQQAMKGYHAMMVDIAVQLVQKWERLNADEHIEVPEDMTRLTLDTIGLCGFNYRFNSFYRDQPHPFITSMVRALDEAMNKLQRANPDDPAYDENKRQFQEDIKVMNDLVDKIIADRKASGEQSDDLLTHMLNGKDPETGEPLDDENIRYQIITFLIAGHETTSGLLSFALYFLVKNPHVLQKAAEEAARVLVDPVPSYKQVKQLKYVGMVLNEALRLWPTAPAFSLYAKEDTVLGGEYPLEKGDELMVLIPQLHRDKTIWGDDVEEFRPERFENPSAIPQHAFKPFGNGQRACIGQQFALHEATLVLGMMLKHFDFEDHTNYELDIKETLTLKPEGFVVKAKSKKIPLGGI'

In [11]:
m.sequence(chainId=['B'])

'TIKEMPQPKTFGELKNLPLLNTDKPVQALMKIADELGEIFKFEAPGRVTRYLSSQRLIKEACDESRFDKNLSQALKFVRDFAGDGLFTSWTHEKNWKKAHNILLPSFSQQAMKGYHAMMVDIAVQLVQKWERLNADEHIEVPEDMTRLTLDTIGLCGFNYRFNSFYRDQPHPFITSMVRALDEAMNKLQRANPDDPAYDENKRQFQEDIKVMNDLVDKIIADRKASGEQSDDLLTHMLNGKDPETGEPLDDENIRYQIITFLIAGHETTSGLLSFALYFLVKNPHVLQKAAEEAARVLVDPVPSYKQVKQLKYVGMVLNEALRLWPTAPAFSLYAKEDTVLGGEYPLEKGDELMVLIPQLHRDKTIWGDDVEEFRPERFENPSAIPQHAFKPFGNGQRACIGQQFALHEATLVLGMMLKHFDFEDHTNYELDIKETLTLKPEGFVVKAKSKKIPLGGI'